In [1]:
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
from args_parameter import args
from PrepareData import ACCESS_BARRA_v1
import xarray as xr

import torch,os,torchvision
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim

# from PIL import Image
import time
from sklearn.model_selection import StratifiedShuffleSplit
import model
from tqdm import tqdm
import math

file_ACCESS_dir="/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
file_BARRA_dir="/g/data/ma05/BARRA_R/analysis/acum_proc/"
# ensemble=['e01','e02']
ensemble=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']
access_rgb_mean= 2.9067910245780248e-05*86400

leading_time=217
leading_time_we_use=31


init_date=date(1970, 1, 1)
start_date=date(1990, 1, 2)
end_date=date(1990,12,30) #if 929 is true we should substract 1 day
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
print(access_rgb_mean)

2.5114674452354135


In [2]:
#for debuging on my computer
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
from args_parameter import args
from PrepareData import ACCESS_BARRA_v1

import torch,os,torchvision
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim

# from PIL import Image
import time
from sklearn.model_selection import StratifiedShuffleSplit
import model
import utility
from tqdm import tqdm
import math
import xarray as xr

file_ACCESS_dir="F:/climate/access-s1/pr/daily/"
file_BARRA_dir="C:/Users/JIA059/barra/"

# ensemble=['e01','e02']
ensemble=['e01','e02']
access_rgb_mean= 2.9067910245780248e-05*86400

leading_time=217
leading_time_we_use=31


init_date=date(1970, 1, 1)
start_date=date(1990, 1, 2)
end_date=date(2018,12,31) #if 929 is true we should substract 1 day
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
print(access_rgb_mean)

2.5114674452354135


In [3]:
train_transforms = transforms.Compose([
#     transforms.Resize(IMG_SIZE),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
    transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
])

data_set=ACCESS_BARRA_v1(start_date,end_date,transform=train_transforms)
train_data,test_data=random_split(data_set,[int(len(data_set)*0.8),len(data_set)-int(len(data_set)*0.8)])


len(train_data)
# len(test_data)

492

In [4]:
train_dataloders =DataLoader(train_data,
                                        batch_size=8,
                                        shuffle=False)
test_dataloders =DataLoader(test_data,
                                        batch_size=8,
                                        shuffle=False)



In [5]:
args.cpu=True
# args.pre_train =False
# args.pre_train ="C:/Users/JIA059/climate_v1_csiro/High-resolution-seasonal-climate-forecast_v1_csiro/model/RCAN_BIX"+str(args.scale[0])+".pt"
# "C:/Users/JIA059/climate_v1_csiro/High-resolution-seasonal-climate-forecast_v1_csiro/model"
def prepare( l, volatile=False):
    device = torch.device('cpu' if args.cpu else 'cuda')
    def _prepare(tensor):
        if args.precision == 'half': tensor = tensor.half()
        return tensor.to(device)

    return [_prepare(_l) for _l in l]

checkpoint = utility.checkpoint(args)
net = model.Model(args, checkpoint).double()

criterion = nn.L1Loss()
optimizer_my = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)


Making model...
accesss-s1 mean (0.4690, 0.4490, 0.4036)


In [ ]:
#training
training_name="temp01"

max_error=10000
for e in range(args.epochs):
    start=time.time()
    if e % 10 == 0:
        for p in optimizer_my.param_groups:
            p['lr'] *= 0.9

    for batch, (lr, hr,_,_) in enumerate(train_dataloders):
    #     print(batch, (lr.size(), hr.size()))
        lr, hr = prepare([lr, hr])
        optimizer_my.zero_grad()
        sr = net(lr, 0)
        error = criterion(sr[:,:,:,0:403], hr)
        if error<max_error:
            max_error=error
            torch.save(net,"C:/Users/JIA059/climate_v1_csiro/High-resolution-seasonal-climate-forecast_v1_csiro/model"+str(e)+".pkl")
#             if not os.path.exists("./model/save/temp01/"):
#                 os.mkdir("./model/save/temp01/")
#             torch.save(net,"./model/save/"+training_name+"/"+str(e)+".pkl")

        error.backward()
        optimizer_my.step()
        break
#     print("epoche: %d,time cost %f s, lr: %f, error: %f"%(e,time.time()-start,optimizer_my.state_dict()['param_groups'][0]['lr'],error.item()))
        
    break
    


In [ ]:
print(lr.shape,hr.shape)

In [6]:
#testing,and evaluation
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr,compare_mse





net.eval()
start=time.time()
with torch.no_grad():
    eval_acc=0
    tqdm_test = tqdm(test_dataloders, ncols=80)
    for idx_img, (lr, hr,date,_) in enumerate(tqdm_test):
        print(date)
        lr, hr = prepare([lr, hr])
        sr = net(lr, 0)
        print(compare_psnr(sr,hr,hr.max()-hr.min()))
#         hr_numpy=hr.numpy()[][0]
#         hr_numpy=np.ascontiguousarray(hr_numpy.transpose((1, 0)))
#         data_hr=xr.DataArray(hr_numpy,coords=[data_set.lon.data,data_set.lat.data],dims=["lat","lon"])
        
#         print(date)
#         dpt.draw_aus(data_hr,
#              colormap = plt.cm.get_cmap('RdBu_r', 10),
#              title="super resolution we predicted",
#              save=False)
        
        
#         sr = utility.quantize(sr, self.args.rgb_range)

        break
print("time cost: %f s "%(start-time.time()))

ModuleNotFoundError: No module named 'skimage'

In [ ]:
print(sr.shape,hr.shape)
# calc_psnr(sr,hr,4,1200)
# data_set.lon.data

In [ ]:
# demo:



with torch.no_grad():
    import time
    start=time.time()
    print("test time for sigle day %s " %(time.time()-start))

    sr_numpy=sr.numpy()[7][0]
#     sr_numpy=np.ascontiguousarray(sr_numpy.transpose((1, 0)))
    data_sr=xr.DataArray(sr_numpy,coords=[data_set.lat.data,data_set.lon.data],dims=["lat","lon"])
    
    hr_numpy=hr.numpy()[7][0]
#     hr_numpy=np.ascontiguousarray(hr_numpy.transpose((1, 0)))
    data_hr=xr.DataArray(hr_numpy,coords=[data_set.lat.data,data_set.lon.data],dims=["lat","lon"])
    print(date)
    dpt.draw_aus(data_hr,
                 colormap = plt.cm.get_cmap('RdBu_r', 10),
                 title="super resolution we predicted",
                 save=False)
    dpt.draw_aus(data_sr,
                 colormap = plt.cm.get_cmap('RdBu_r', 10),
                 title="super resolution we predicted",
                 save=False)

# psnr1(sr, hr)
# print(hr.max())

In [ ]:
print(hr_numpy.shape)
print(data_set.lon.data.shape)
print(data_set.lat.data.shape)

In [ ]:
#testing



epoch = self.scheduler.last_epoch + 1
self.ckp.write_log('\nEvaluation:')
self.ckp.add_log(torch.zeros(1, len(self.scale)))
self.model.eval()

timer_test = utility.timer()
with torch.no_grad():
    for idx_scale, scale in enumerate(self.scale):
        eval_acc = 0
        self.loader_test.dataset.set_scale(idx_scale)
        tqdm_test = tqdm(self.loader_test, ncols=80)
        for idx_img, (lr, hr, filename, _) in enumerate(tqdm_test):
            filename = filename[0]
            no_eval = (hr.nelement() == 1)
            if not no_eval:
                lr, hr = self.prepare([lr, hr])
            else:
                lr = self.prepare([lr])[0]

            sr = self.model(lr, idx_scale)
            sr = utility.quantize(sr, self.args.rgb_range)

            save_list = [sr]
            if not no_eval:
                eval_acc += utility.calc_psnr(
                    sr, hr, scale, self.args.rgb_range,
                    benchmark=self.loader_test.dataset.benchmark
                )
                save_list.extend([lr, hr])

            if self.args.save_results:
                self.ckp.save_results(filename, save_list, scale)

        self.ckp.log[-1, idx_scale] = eval_acc / len(self.loader_test)
        best = self.ckp.log.max(0)
        self.ckp.write_log(
            '[{} x{}]\tPSNR: {:.3f} (Best: {:.3f} @epoch {})'.format(
                self.args.data_test,
                scale,
                self.ckp.log[-1, idx_scale],
                best[0][idx_scale],
                best[1][idx_scale] + 1
            )
        )

self.ckp.write_log(
    'Total time: {:.2f}s\n'.format(timer_test.toc()), refresh=True
)
if not self.args.test_only:
    self.ckp.save(self, epoch, is_best=(best[1][0] + 1 == epoch))

In [ ]:
torch.new()
np.random.rand(8, 4, 78, 100)

In [ ]:
# channel=1
# x=np.random.rand(8,channel, 78, 100)
# x=x.astype(np.float32)
# print(x.dtype)

# means, stdevs = [], []

# for i in range(channel):
#     pixels = x[:, i, :, :].ravel()  # 拉成一行
#     means.append(np.mean(pixels))
#     stdevs.append(np.std(pixels))

# x=torch.tensor(x)


# # std=np.std(x.ravel(),axis=0)
# # print(std.shape)
# # mean=np.mean(x)
# print(means)
# print(stdevs)

# # x=torch.rand((8, 3, 78, 100))
# # std=torch.std(x)
# # mean=torch.mean(x)

# # x.dtype="float32"

# class MeanShift(nn.Conv2d):
#     def __init__(self, rgb_range, rgb_mean, rgb_std,channels, sign=-1):
#         super(MeanShift, self).__init__(channel, channel, kernel_size=1)
#         std = torch.Tensor(rgb_std)
#         self.weight.data = torch.eye(channel).view(channel, channel, 1, 1)
#         self.weight.data.div_(std.view(channel, 1, 1, 1))
#         self.bias.data = sign * rgb_range * torch.Tensor(rgb_mean)
#         self.bias.data.div_(std)
#         self.requires_grad = False
        
# aa=MeanShift(1,means,stdevs,3)
# # aa(x)
# np.mean(aa(x).detach().numpy())


In [ ]:
# # checkpoint = utility.checkpoint(args)
# # net = model.Model(args, checkpoint)
# # criterion = nn.L1Loss()
# # optimizer_my = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
# def prepare( l, volatile=False):
#     device = torch.device('cpu' if args.cpu else 'cuda')
#     def _prepare(tensor):
#         if args.precision == 'half': tensor = tensor.half()
#         return tensor.to(device)

#     return [_prepare(_l) for _l in l]

# max_error=10000
# for e in range(args.epochs):
#     if e % 10 == 0:
#         for p in optimizer_my.param_groups:
#             p['lr'] *= 0.9
        
#     net.train()
#     for batch, (lr, hr) in enumerate(train_dataloders):
#         lr, hr = prepare([lr, hr])
# #         print(lr)

#         optimizer_my.zero_grad()
#         sr = net(lr, 0)
# #         sr=utility.fit_size(sr,args)

#         error = criterion(sr[:,:,:,0:403], hr)
#         if error<max_error:
#             max_error=error
#             torch.save(net,"C:/Users/JIA059/climate_v1_csiro/High-resolution-seasonal-climate-forecast_v1_csiro/model/save/"+str(e)+".pkl")
#         error.backward()
#         optimizer_my.step()
#     print("epoche: %d, lr: %f, error: %f"%(e,optimizer_my.state_dict()['param_groups'][0]['lr'],error.item()))